In [31]:
import pandas as pd
import numpy as np
import re

In [10]:
gp_mapped = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/UKBB_diagnoses_Jun2025/gpmappedicd10_analytical.csv").iloc[:,1:]

In [11]:
gp_mapped.rename(columns={"event_dt":"date","icd10_analytical":"code_mapped"},inplace=True)

In [13]:
gp_mapped["source"] = "GP"

In [14]:
gp_mapped

,eid,date,icd10,code_mapped,source
0,1376547,2010-01-13,A02,A020,GP
1,2650372,2014-08-19,K64,K648,GP
2,1433988,2009-12-16,J22,J22X,GP
3,3231581,2008-01-02,B02,B02,GP
4,5088063,2010-09-29,J01,J01,GP
...,...,...,...,...,...
2475135,1214117,2010-01-20,D17,D172,GP
2475136,5228770,2014-11-26,H35,H353,GP
2475137,3995978,2002-08-14,M25,M256,GP
2475138,2223894,1999-01-01,M79,M796,GP


In [4]:
df_final = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/UKBB_diagnoses_Jun2025/processed/Recoded_all_diagnoses_ICD10.csv")

/var/tmp/pbs.1546476.pbs/ipykernel_4065871/2310557967.py:1: DtypeWarning: Columns (1,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/UKBB_diagnoses_Jun2025/processed/Recoded_all_diagnoses_ICD10.csv")


In [17]:
df_final

,eid,code,date,source,description,code_mapped,norm_description,icd10_bert,confidence
0,3738132,F419,2019-12-20,ICD10,NaN,NaN,NaN,NaN,NaN
1,3738132,H023,2012-08-20,ICD10,NaN,NaN,NaN,NaN,NaN
2,3738132,H024,2012-08-20,ICD10,NaN,NaN,NaN,NaN,NaN
3,3738132,H269,2020-11-19,ICD10,NaN,NaN,NaN,NaN,NaN
4,3738132,H348,2019-08-20,ICD10,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
8690904,5543393,1676,NaN,Selfreport,yellow fever,A95,yellow fever,NaN,NaN
8690905,3529284,1676,1972-09-13,Selfreport,yellow fever,A95,yellow fever,NaN,NaN
8690906,1781746,1676,1952-09-13,Selfreport,yellow fever,A95,yellow fever,NaN,NaN
8690907,1121296,1676,1950-10-19,Selfreport,yellow fever,A95,yellow fever,NaN,NaN


In [18]:
df_all = pd.concat([df_final, gp_mapped])

In [21]:
df_all["code_mapped"] = df_all["code_mapped"].fillna(df_all["code"])

In [23]:
df_all["code_mapped"].isna().sum()

0

In [25]:
df_all = df_all[["eid","code_mapped","date","source"]]

In [52]:
def clean_code(value):
    # Convert to string and remove punctuation (keep only letters and numbers)
    value = re.sub(r'[^A-Za-z0-9]', '', str(value))

    # If 3-character ICD10 + ends with X (e.g., A12X) → remove final X
    if re.match(r'^[A-Za-z0-9]{3}X$', value):
        value = value[:-1]
    else:
        # Remove all X except if it's the first character
        if value.startswith('X'):
            value = 'X' + value[1:].replace('X', '')
        else:
            value = value.replace('X', '')

    return value

In [53]:
df_all["code_mapped"] = df_all["code_mapped"].apply(clean_code)

/var/tmp/pbs.1546476.pbs/ipykernel_4065871/4115122075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all["code_mapped"] = df_all["code_mapped"].apply(clean_code)


In [54]:
df_all

,eid,code_mapped,date,source
0,3738132,F419,2019-12-20,ICD10
1,3738132,H023,2012-08-20,ICD10
2,3738132,H024,2012-08-20,ICD10
3,3738132,H269,2020-11-19,ICD10
4,3738132,H348,2019-08-20,ICD10
...,...,...,...,...
2475135,1214117,D172,2010-01-20,GP
2475136,5228770,H353,2014-11-26,GP
2475137,3995978,M256,2002-08-14,GP
2475138,2223894,M796,1999-01-01,GP


In [55]:
codings = pd.read_excel("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/codings/all_lkps_maps_v4.xlsx", sheet_name="icd10_lkp")

In [56]:
codings = codings[["ICD10_CODE", "ALT_CODE","DESCRIPTION"]]

In [57]:
codings["ICD10_CODE"] = codings["ICD10_CODE"].apply(clean_code)
codings["ALT_CODE"] = codings["ALT_CODE"].apply(clean_code)


In [58]:
codings[codings["ICD10_CODE"] != codings["ALT_CODE"]]

,ICD10_CODE,ALT_CODE,DESCRIPTION
4026,I700,I7000,Atherosclerosis of aorta
4027,I700,I7001,Atherosclerosis of aorta
4029,I701,I7010,Atherosclerosis of renal artery
4030,I701,I7011,Atherosclerosis of renal artery
4032,I702,I7020,Atherosclerosis of arteries of extremities
...,...,...,...
12803,T12,T120,"Fracture of lower limb, level unspecified"
12804,T12,T121,"Fracture of lower limb, level unspecified"
12819,T142,T1420,Fracture of unspecified body region
12820,T142,T1421,Fracture of unspecified body region


In [59]:
codings

,ICD10_CODE,ALT_CODE,DESCRIPTION
0,A00,A00,Cholera
1,A000,A000,"Cholera due to Vibrio cholerae 01, biovar chol..."
2,A001,A001,"Cholera due to Vibrio cholerae 01, biovar eltor"
3,A009,A009,"Cholera, unspecified"
4,A01,A01,Typhoid and paratyphoid fevers
...,...,...,...
17931,U848,U848,Resistance to other specified antimicrobial drug
17932,U849,U849,Resistance to unspecified antimicrobial drugs
17933,U85,U85,Resistance to antineoplastic drugs
17934,nan,nan,NaN


In [63]:
coding_19 = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/codings/coding19.tsv", sep="\t")

In [88]:
df_blocks = coding_19[coding_19["coding"].str.startswith("Block")]

In [89]:
df_blocks[['start', 'end']] = df_blocks['coding'].str.extract(r'([A-Z]\d{2})-([A-Z]\d{2})')

/var/tmp/pbs.1546476.pbs/ipykernel_4065871/2154546066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_blocks[['start', 'end']] = df_blocks['coding'].str.extract(r'([A-Z]\d{2})-([A-Z]\d{2})')
/var/tmp/pbs.1546476.pbs/ipykernel_4065871/2154546066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_blocks[['start', 'end']] = df_blocks['coding'].str.extract(r'([A-Z]\d{2})-([A-Z]\d{2})')


In [93]:
df_blocks

,coding,meaning,node_id,parent_id,selectable,start,end
921,Block A00-A09,A00-A09 Intestinal infectious diseases,23,1,N,A00,A09
922,Block A15-A19,A15-A19 Tuberculosis,24,1,N,A15,A19
923,Block A20-A28,A20-A28 Certain zoonotic bacterial diseases,25,1,N,A20,A28
924,Block A30-A49,A30-A49 Other bacterial diseases,26,1,N,A30,A49
925,Block A50-A64,A50-A64 Infections with a predominantly sexual...,27,1,N,A50,A64
...,...,...,...,...,...,...,...
1179,Block Z30-Z39,Z30-Z39 Persons encountering health services i...,279,21,N,Z30,Z39
1180,Block Z40-Z54,Z40-Z54 Persons encountering health services f...,280,21,N,Z40,Z54
1181,Block Z55-Z65,Z55-Z65 Persons with potential health hazards ...,281,21,N,Z55,Z65
1182,Block Z70-Z76,Z70-Z76 Persons encountering health services i...,282,21,N,Z70,Z76


In [127]:
block_dict = dict(zip(df_blocks["coding"], df_blocks["meaning"]))


In [94]:
def find_block(code, df_blocks):
    letter = code[0]
    num = int(code[1:])

    for _, row in df_blocks.iterrows():
        start_letter, start_num = row['start'][0], int(row['start'][1:])
        end_letter, end_num = row['end'][0], int(row['end'][1:])
        
        if letter == start_letter == end_letter and start_num <= num <= end_num:
            return row['coding']
    return None

In [84]:
df_all["coding"] = df_all["code_mapped"].str[:3]
df_merged = pd.merge(df_all, coding_19, on="coding")
df_merged["date"] = pd.to_datetime(df_merged["date"])
df_merged["earliest_2_code_diagnosis"] = df_merged.groupby(["eid", "coding"])["date"].transform("min")


/var/tmp/pbs.1546476.pbs/ipykernel_4065871/90286784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all["coding"] = df_all["code_mapped"].str[:3]


In [115]:
codings_mapped = df_merged["coding"].unique().tolist()

In [117]:
len(codings_mapped)

2002

In [118]:
coding_to_block = {
    code: find_block(code, df_blocks)
    for code in codings_mapped
}

In [121]:
df_merged.drop(columns=["node_id","parent_id","selectable"],inplace=True)

In [123]:
df_merged["block"] = df_merged["coding"].map(coding_to_block)

In [125]:
df_merged["earliest_blockcode_diagnosis"] = df_merged.groupby(["eid", "block"])["date"].transform("min")


In [128]:
df_merged["meaning_block"]  = df_merged["block"].map(block_dict)

In [132]:
df_merged.to_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/UKBB_diagnoses_Jun2025/processed/ICD10_mapped_all.csv", index=False)